## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import pandas as pd
from datetime import datetime
from prophet.serialize import model_from_json
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
# Retrieve the 'medical_features' feature view
feature_view = fs.get_feature_view(
    name='medical_features',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
# Get the model registry
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch and test the model</span>

Finally you can start making predictions with your model!

Retrieve your model from Hopsworks model registry.

In [ ]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="waiting_time_forecast_model",
    version=1,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
with open(saved_model_dir + '/serialized_model.json', 'r') as fin:
    model = model_from_json(fin.read())  # Load model

## <span style="color:#ff5f27;">🔮  Batch Prediction </span>


In [ ]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get the batch data
batch_data = feature_view.get_batch_data(
    start_time=datetime(2015, 10, 19), 
    end_time=datetime(2017, 12, 29), 
    event_time=True,
)

batch_data['ds'] = batch_data.date
batch_data['ds'] = pd.to_datetime(batch_data.ds)
batch_data['ds'] = batch_data.ds.map(lambda x: x.replace(tzinfo=None))
batch_data.drop(columns=["date"], axis=1, inplace=True)
batch_data = batch_data.sort_values("ds")

# Display the first 3 rows
batch_data.head(3)

In [ ]:
# use the model to make a forecast
forecast = model.predict(batch_data)

# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)

---